In [23]:
import pandas as pd
import numpy as np
import altair as alt
from scipy.optimize import minimize

from true_skill_through_time import *

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

This notebook does essentially the same thing as the `true_skill_wc3` notebook, except it does it via the `TrueSkillThroughTimeApplied` class.

The class is to be used on more sports/esports, outside of Warcraft3.

In [24]:
games_raw = pd.read_csv('data\warcraft3.csv').query('(competitor_1_score > -0.0001) & (competitor_2_score > -0.0001)').iloc[-50000:]
games_raw['timestamp'] = pd.to_datetime(games_raw['date'])

In [25]:
game_composition = []
times = []
np.random.seed(0)

for _, row in games_raw.iterrows():
    c1, c2, c1s, c2s, t = row['competitor_1'], row['competitor_2'], row['competitor_1_score'], row['competitor_2_score'], row['timestamp']

    assert c1s == int(c1s)
    assert c2s == int(c2s)
    
    comp = [(c1, c2)] * int(c1s) + [(c2, c1)] * int(c2s)
    comp = np.random.permutation(comp).tolist() # Game order matters and we don't actually know it, so we randomize over it.

    for cp_g in comp:
        game_composition.append(cp_g)
        times.append(t)

games = pd.DataFrame(game_composition, columns=['winner', 'loser']).assign(timestamp = times)
games.head()

,winner,loser,timestamp
0,TeRRoR,Ark,2020-05-10 00:00:36
1,Rigety,MisterWinner,2020-05-10 00:00:37
2,Paladyn,SaturN,2020-05-10 00:00:38
3,Blade,TimberOwls,2020-05-10 00:00:39
4,HawK,GodFather,2020-05-10 00:00:40


In [26]:
self = TrueSkillThroughTimeApplied(games)

In [27]:
print(games.head().to_string())

    winner         loser           timestamp  time_0_to_999_int
0   TeRRoR           Ark 2020-05-10 00:00:36                  0
1   Rigety  MisterWinner 2020-05-10 00:00:37                  0
2  Paladyn        SaturN 2020-05-10 00:00:38                  0
3    Blade    TimberOwls 2020-05-10 00:00:39                  0
4     HawK     GodFather 2020-05-10 00:00:40                  0


In [28]:
optimal_gamma =  0.08105500545076469
optimal_sigma = 3.738460608607473
optimal_beta = 2.7614029558670308

self.set_optimal_parameters(gamma = optimal_gamma, sigma = optimal_sigma, beta = optimal_beta)

In [29]:
top_players_elo = ['Happy', 'Lyn', 'Fortitude', 'Eer0', 'Sok', 'ColorFul', 'Kaho', 'FoCuS', 'Moon',  'LabyRinth', 'Life', 'LawLiet', 'Blade', 'Fly100%', 'Starbuck', 'Soin', 'Infi', 'Sini',  'Dise', 'Chaemiko', 'EleGaNt', 'Leon', 'Lin_Guagua', 'WFZ', 'XiaoKK', 'XlorD', 'ReMinD', 'PaTo', 'HawK']

In [30]:
skill_curves = self.set_skill_curves()

In [31]:
self.plot_player_skills(players = top_players_elo[:5])

alt.LayerChart(...)

In [32]:
self.plot_calibration()

alt.LayerChart(...)